# Fine-tune TinyGPT on Your Own Text
Make it speak in your style (poems, stories, etc.).


In [ ]:
# Bonus fine-tune notebook
# Assumes `model`, `encode`, `decode`, `block_size`, and `text`, `stoi` from 02_tinyGPT have been run.
import torch, textwrap
from tqdm.auto import tqdm

student_text = \"\"\"
Once upon a time in Puteaux, a curious coder started a tiny GPT...
\"\"\"

repeat = 50
mix = text + "\n\n" + student_text*repeat
mix_ids = torch.tensor([stoi.get(c, 0) for c in mix], dtype=torch.long)

n2 = int(0.9*len(mix_ids)); train2, val2 = mix_ids[:n2], mix_ids[n2:]
def get_batch2(split):
    src = train2 if split=="train" else val2
    ix = torch.randint(len(src)-block_size-1, (64,))
    x = torch.stack([src[i:i+block_size] for i in ix])
    y = torch.stack([src[i+1:i+block_size+1] for i in ix])
    return x.to(next(model.parameters()).device), y.to(next(model.parameters()).device)

opt = torch.optim.AdamW(model.parameters(), lr=1e-4)
for step in range(600):
    xb,yb = get_batch2("train")
    _,loss = model(xb,yb)
    opt.zero_grad(set_to_none=True); loss.backward(); opt.step()

def sample_text(prompt="Once upon a time", temperature=0.9, top_k=50):
    start = encode(prompt).unsqueeze(0).to(next(model.parameters()).device)
    out = model.generate(start, max_new_tokens=400, temperature=temperature, top_k=top_k)[0].tolist()
    return decode(out)

print(textwrap.fill(sample_text(), width=90))
